In [1]:
import json
import re
import pandas as pd

In [2]:
f = open("all_scripts_raw.json", "r")
data = json.load(f)

In [3]:
print(data.keys())
df = pd.DataFrame(data)
df.head()

dict_keys(['DS9', 'TOS', 'TAS', 'TNG', 'VOY', 'ENT'])


,DS9,TOS,TAS,TNG,VOY,ENT
episode 0,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - The Ca...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\nThe Voyager Transcripts - Caretaker\...,\n\n\n\n\n\nThe Enterprise Transcripts - Broke...
episode 1,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - The Ma...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\n\nThe Voyager Transcripts - Parallax...,\n\n\n\n\n\nThe Enterprise Transcripts - Fight...
episode 2,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - Charli...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\n\nThe Voyager Transcripts - Time and...,\n\n\n\n\n\nThe Enterprise Transcripts - Stran...
episode 3,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - Where ...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\n\n\nThe Voyager Transcripts - Phage\...,\n\n\n\n\n\nThe Enterprise Transcripts - Unexp...
episode 4,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - The Na...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\n\nThe Voyager Transcripts - The Clou...,\n\n\n\n\n\nThe Enterprise Transcripts - Terra...


In [4]:
for series in df:
    for ep in df[series].index:
        if type(df[series][ep]) == str:
            df[series][ep] = re.sub(r'\s+', ' ', df[series][ep].strip())

# df['DS9'][0]

In [5]:
df_scenes = df.copy()

In [6]:
s = "This is a regex test [bracket] other text [Ops] is here [test] more text [OC] this is comms [on screen] yo"
scene_regex = re.compile(r'(?=[\[]+(?!OC)+(?!on))')
scene_regex.split(s)

['This is a regex test ',
 '[bracket] other text ',
 '[Ops] is here ',
 '[test] more text [OC] this is comms [on screen] yo']

In [7]:
for series in df_scenes:
    for ep in df_scenes[series].index:
        if type(df[series][ep]) == str:
            df_scenes[series][ep] = scene_regex.split(df_scenes[series][ep])

In [8]:
df_scenes['DS9'][1][:3]

['The Deep Space Nine Transcripts - Past Prologue Past Prologue Stardate: Unknown Original Airdate: 11 Jan, 1993 ',
 "[Replimat] GARAK: It's Doctor Bashir, isn't it? Of course it is. May I introduce myself? BASHIR: Er, yes, yes, of course. GARAK: My name is Garak. Cardassian by birth, obviously. The only one of us left on this station, as a matter of fact, so I do appreciate making new friends whenever I can. You are new to this station, I believe. BASHIR: I am, yes. Though, though I understand you've been here quite a while. GARAK: Ah, you know of me then. BASHIR: Would you care for some of this Tarkalean tea? It's very good. GARAK: What a thoughtful young man. How nice that we've met. BASHIR: You know, some people say that you remained on DS Nine as the eyes and ears of your fellow Cardassians. GARAK: You don't say? Doctor, you're not intimating that I'm considered some sort of spy, are you? BASHIR: I wouldn't know, sir. GARAK: Ah. An open mind. The essence of intellect. As you may a

In [23]:
class Vocab:
    def __init__(self):
        self.words = []
        self.word_count = []
        self.df = pd.DataFrame(columns=['word', 'count'])

    def add_word(self, word):
        if word in self.words:
            self.word_count[self.words.index(word)] += 1
        else:
            self.words.append(word)
            self.word_count.append(1)

    def get_word_count(self, word):
        if word in self.words:
            return self.word_count[self.words.index(word)]
        else:
            return 0

    def rebuild_dataframe(self):
        self.df = pd.DataFrame({"word": vocab.words, "count": vocab.word_count})
        # for i in range(len(self.words)):
        #     self.df.loc[i] = [self.words[i], self.word_count[i]]
        self.df.sort_values('count', ascending=False, inplace=True)

    def get_as_dataframe(self):
        return self.df

    def get_total_size(self):
        return sum(self.word_count)

    def __len__(self):
        return len(self.words)

    def get_words(self, sorted=True):
        # if sorted:
        #     return self.words.sort_values('count', ascending=False)
        # else:
        return self.words
    

In [57]:
split_words = re.compile(r'[\s\[\]\(\),.?!:]')

In [210]:
vocab = Vocab()

print("Loading words...")

for series in df:
    for ep in df[series]:
        if type(ep) != str:
            continue
        word_split = re.split(r'[\s\[\]\(\),.?!:]', ep.strip())
        words = [w.strip() for w in word_split if w != '' and not w.isdigit()]
        nums = [w for w in word_split if w.isdigit()]
        digits = []
        for n in nums:
            l = re.split(r'(\d)', n)
            for d in l:
                if d.isdigit():
                    digits.append(d)

        for w in words:
            vocab.add_word(w)

        for w in digits:
            vocab.add_word(w)
    print(f"Finished loading {series}")

print("Unique words:", len(vocab))
print("Generating Dataframe...")
vocab.rebuild_dataframe()
vocab.get_as_dataframe().to_csv('all_vocab.csv')
print("Done!")

# vocab.get_as_dataframe().head()
# re.split(r'[\s\[\]\(\),.?!:]+', df_scenes['DS9'][1][1].strip())

Loading words...
Finished loading DS9
Finished loading TOS
Finished loading TAS
Finished loading TNG
Finished loading VOY
Finished loading ENT
Unique words: 58581
Generating Dataframe...
Done!


In [21]:
len(vocab.words), len(vocab.word_count)
voc_df = pd.DataFrame({"words": vocab.words, "counts": vocab.word_count})

In [27]:
voc_df.sort_values('counts', ascending=False, inplace=True)
voc_df.head()
voc_df.to_csv("vocab.csv")

In [196]:
vl = len(voc_df[voc_df['counts'] > 1])
voc_df_u = voc_df[voc_df['counts'] > 1]
voc_df_u.reset_index(inplace=True)
print(vl)

37182


In [202]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
tf.keras.backend.clear_session()

In [155]:
def tokenize(word, vocab_df):
    if word in vocab_df['words'].to_list():
        idx = vocab_df[vocab_df['words'] == word].index[0]
        arr = np.zeros(len(vocab_df))
        arr[idx] = 1.
        return arr
    else:
        return np.zeros(len(vocab_df))

In [198]:
class WordFromVocab(Sequence):
    def __init__(self, vocab_df, batch_size):
        self.vocab_df = vocab_df
        self.batch_size = batch_size
        self.indices = np.arange(len(vocab_df))

    def __len__(self):
        return len(self.vocab_df) // self.batch_size

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = (idx + 1) * self.batch_size
        arr = np.array([tokenize(w, self.vocab_df) for w in self.vocab_df['words'][start:end]], dtype=np.float32)
        
        return arr, arr

    def on_epoch_end(self):
        np.random.shuffle(self.indices)